# Web Scraping: CNMV

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
import os

---
## Scraping de la CNMV

### ¿Qué es la CNMV?

La **Comisión Nacional del Mercado de Valores** (cnmv.es) supervisa los mercados financieros en España. Su web contiene registros públicos con información sobre:

- Sociedades y Agencias de Valores registradas
- Empresas de Asesoramiento Financiero
- Entidades advertidas ("chiringuitos financieros")
- Hechos relevantes de empresas cotizadas

### Objetivo

Vamos a extraer el **listado de Sociedades y Agencias de Valores** registradas en la CNMV, obteniendo:
- Nombre de la entidad
- Número de registro
- Fecha de registro
- Dirección

### 1.1 Descargar la página del listado

In [ ]:
# URL del listado de Sociedades y Agencias de Valores
url_cnmv = "https://www.cnmv.es/portal/consultas/listadoentidad?id=1&tipoent=0&lang=es"

# La CNMV necesita headers para responder correctamente
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36",
    "Accept-Language": "es-ES,es;q=0.9",
}

respuesta = requests.get(url_cnmv, headers=headers)
print(f"Status code: {respuesta.status_code}")
print(f"Tamaño: {len(respuesta.text):,} caracteres") # Las {} son marcadores de expresión dentro del f-string

In [ ]:
soup = BeautifulSoup(respuesta.text, "html.parser")

# Confirmamos que llegamos a la página correcta
titulo = soup.find("h1")
if titulo:
    print("Página:", titulo.text.strip())
else:
    print("Título de la página:", soup.title.text.strip() if soup.title else "no encontrado")

### 1.2 Analizar la estructura HTML

Antes de extraer datos, necesitamos **inspeccionar** el HTML para entender cómo están organizados.

> En tu navegador, haz clic derecho sobre un nombre de entidad → "Inspeccionar" para ver los tags HTML.

La página de la CNMV tiene bloques repetidos con esta estructura:
```
NOMBRE DE LA ENTIDAD
Número y fecha de registro oficial: 251 - 29/08/2013
Dirección: CALLE TAL, Nº X - CÓDIGO CIUDAD
```

Vamos a buscar los elementos que contienen esta información.

In [ ]:
# Buscamos el contenedor principal del contenido
# La CNMV usa ASP.NET, el contenido suele estar en un div con id "maincontent" o similar
main = soup.find("div", id="maincontent") or soup.find("main") or soup

# Exploramos: buscamos todos los textos que contengan "Número y fecha de registro"
# Esto nos ayuda a localizar los bloques de entidades
textos_registro = main.find_all(string=re.compile(r"Número y fecha de registro"))
print(f"Bloques con 'Número y fecha de registro' encontrados: {len(textos_registro)}")

# Contexto HTML para entender la estructura
if textos_registro:
    # Subimos al elemento padre para ver el bloque completo
    bloque = textos_registro[0].find_parent("div") or textos_registro[0].parent
    print("\nEstructura del primer bloque:")
    print(bloque.prettify()[:300])

### 1.3 Extraer los datos de cada entidad

Ahora que conocemos la estructura, vamos a extraer los datos de forma sistemática.

> **Nota:** Las webs institucionales pueden cambiar su estructura HTML sin aviso. Si los selectores no funcionan, habrá que inspeccionarla de nuevo. Esto es parte de la realidad del scraping.

In [ ]:
# Extraemos el texto completo del contenido principal
texto_completo = main.get_text(separator="\n")

# Usamos expresiones regulares para capturar los patrones de datos
# Patrón: líneas con "Número y fecha de registro oficial: NUM - DD/MM/AAAA"
patron_registro = re.compile(
    r"Número y fecha de registro oficial:\s*(\d+)\s*-\s*(\d{2}/\d{2}/\d{4})"
)

# Patrón: líneas con "Dirección: ..."
patron_direccion = re.compile(
    r"Dirección:\s*(.+)"
)

registros = patron_registro.findall(texto_completo)
direcciones = patron_direccion.findall(texto_completo)

print(f"Registros encontrados: {len(registros)}")
print(f"Direcciones encontradas: {len(direcciones)}")

# Mostramos los primeros 3
for num, fecha in registros[:3]:
    print(f"  Nº {num} - Fecha: {fecha}")

In [ ]:
# Ahora extraemos los nombres de las entidades
# Los nombres aparecen justo ANTES de cada "Número y fecha de registro"
# Dividimos el texto por ese patrón y cogemos la línea anterior

lineas = texto_completo.split("\n")
lineas = [l.strip() for l in lineas if l.strip()]  # Limpiamos vacías

entidades = []
for i, linea in enumerate(lineas):
    match = patron_registro.search(linea)
    if match:
        # El nombre de la entidad suele estar en la línea anterior
        nombre = lineas[i - 1] if i > 0 else "Desconocido"
        num_registro = match.group(1)
        fecha_registro = match.group(2)
        
        # La dirección suele estar en la línea siguiente
        direccion = ""
        if i + 1 < len(lineas):
            match_dir = patron_direccion.search(lineas[i + 1])
            if match_dir:
                direccion = match_dir.group(1).strip()
        
        entidades.append({
            "nombre": nombre,
            "num_registro": int(num_registro),
            "fecha_registro": fecha_registro,
            "direccion": direccion,
        })

print(f"{len(entidades)} entidades extraídas")
print("\nPrimeras 3:")
for e in entidades[:3]:
    print(f"  {e['nombre']} (Reg. {e['num_registro']}, {e['fecha_registro']})")

In [ ]:
df_cnmv = pd.DataFrame(entidades)
df_cnmv["fecha_registro"] = pd.to_datetime(df_cnmv["fecha_registro"], format="%d/%m/%Y")
df_cnmv = df_cnmv.sort_values("fecha_registro", ascending=False).reset_index(drop=True)
df_cnmv

### 1.4 Análisis rápido

In [ ]:
# ¿Cuántas entidades se registraron por año?
df_cnmv["anyo"] = df_cnmv["fecha_registro"].dt.year
print("Entidades registradas por año (últimos 10):")
print(df_cnmv["anyo"].value_counts().sort_index(ascending=True).tail(10).to_string())

# ¿En qué ciudades están?
# La dirección suele terminar en "CÓDIGO CIUDAD", extraemos la ciudad
df_cnmv["ciudad"] = df_cnmv["direccion"].str.extract(r"\d{5}\s+(.+)$")[0]
print("\nTop 5 ciudades:")
print(df_cnmv["ciudad"].value_counts().head().to_string())

### 2.1 Descargar un informe público

Vamos a trabajar con informes reales y públicos. Usaremos:
- **Iberdrola** — Informe de Gases de Efecto Invernadero 2023
- **Iberdrola** — Indicadores clave de sostenibilidad 2023

> 💡 Estos PDFs son públicos y están enlazados desde la web oficial de Iberdrola.

In [ ]:
# URLs directas a los PDFs públicos de Iberdrola
pdfs = {
    "iberdrola_gei": "https://www.iberdrola.com/documents/20125/41101/informe-gei-2023.pdf",
    "iberdrola_indicadores": "https://www.iberdrola.com/documents/20125/3643974/informe-integrado-esg-2023-indicadores-clave-sostenibilidad.pdf",
}

# Creamos una carpeta para guardar los PDFs
os.makedirs("pdfs_sostenibilidad", exist_ok=True)

# Descargamos cada PDF
for nombre, url in pdfs.items():
    ruta = f"pdfs_sostenibilidad/{nombre}.pdf"
    
    if os.path.exists(ruta):
        print(f"  ✓ {nombre}.pdf ya existe, saltando descarga")
        continue
    
    print(f"  Descargando {nombre}...", end=" ")
    resp = requests.get(url, headers=headers, timeout=60)
    
    if resp.status_code == 200 and resp.headers.get("Content-Type", "").startswith("application/pdf"):
        with open(ruta, "wb") as f:
            f.write(resp.content)
        print(f"OK ({len(resp.content)/1024:.0f} KB)")
    else:
        print(f"ERROR (status {resp.status_code})")

# Verificamos qué tenemos
for f in os.listdir("pdfs_sostenibilidad"):
    size = os.path.getsize(f"pdfs_sostenibilidad/{f}") / 1024
    print(f"  📄 {f} ({size:.0f} KB)")

### 2.2 Explorar la estructura de un PDF

Antes de extraer datos, vamos a explorar qué contiene el PDF: número de páginas, texto y tablas.

In [ ]:
# Abrimos el informe GEI de Iberdrola
pdf = pdfplumber.open("pdfs_sostenibilidad/iberdrola_gei.pdf")

print(f"Número de páginas: {len(pdf.pages)}")
print(f"\n--- Texto de la primera página (primeros 500 caracteres) ---\n")
texto_p1 = pdf.pages[0].extract_text()
print(texto_p1[:500] if texto_p1 else "(sin texto extraíble en esta página)")

In [ ]:
# Buscamos tablas en todas las páginas
print("Tablas encontradas por página:")
total_tablas = 0

for i, pagina in enumerate(pdf.pages):
    tablas = pagina.extract_tables()
    if tablas:
        total_tablas += len(tablas)
        for j, tabla in enumerate(tablas):
            print(f"  Página {i+1}, Tabla {j+1}: {len(tabla)} filas x {len(tabla[0])} columnas")

print(f"\nTotal de tablas: {total_tablas}")

### 2.3 Extraer tablas y convertirlas a DataFrames

`pdfplumber` devuelve las tablas como listas de listas. Podemos convertirlas directamente a DataFrames de pandas.

In [ ]:
# Extraemos TODAS las tablas del PDF en una lista de DataFrames
todos_los_dfs = []

for i, pagina in enumerate(pdf.pages):
    tablas = pagina.extract_tables()
    for j, tabla in enumerate(tablas):
        # La primera fila suele ser la cabecera
        if len(tabla) > 1:
            df_tabla = pd.DataFrame(tabla[1:], columns=tabla[0])
            df_tabla.attrs["fuente"] = f"Página {i+1}, Tabla {j+1}"
            todos_los_dfs.append(df_tabla)

print(f"DataFrames creados: {len(todos_los_dfs)}")

# Mostramos el primero como ejemplo
if todos_los_dfs:
    print(f"\nPrimer DataFrame ({todos_los_dfs[0].attrs.get('fuente', '')}):")
    display(todos_los_dfs[0])

In [ ]:
pdf.close()

### 2.4 Buscar indicadores concretos en el texto del PDF

A veces los datos no están en tablas limpias, sino dispersos en el texto. Podemos usar **expresiones regulares** para buscar indicadores concretos.

Los informes de sostenibilidad suelen usar patrones como:
- `"emisiones de CO2: 125.430 tCO2eq"`
- `"Alcance 1: 118.200 toneladas"`
- `"43.175 MW verdes instalados"`

In [ ]:
# Extraemos TODO el texto del PDF (todas las páginas)
pdf = pdfplumber.open("pdfs_sostenibilidad/iberdrola_gei.pdf")

texto_completo = ""
for pagina in pdf.pages:
    texto = pagina.extract_text()
    if texto:
        texto_completo += texto + "\n"

pdf.close()

print(f"Texto total extraído: {len(texto_completo):,} caracteres")
print(f"Palabras aproximadas: {len(texto_completo.split()):,}")

In [ ]:
# Definimos patrones de búsqueda para indicadores ESG comunes
# Cada patrón captura el número asociado al indicador

patrones_esg = {
    "Emisiones CO2 (tCO2)": r"([\d.,]+)\s*(?:t\s*CO2|tCO2eq|toneladas? de CO)",
    "Alcance 1": r"[Aa]lcance\s*1[^\d]{0,30}([\d.,]+)",
    "Alcance 2": r"[Aa]lcance\s*2[^\d]{0,30}([\d.,]+)",
    "Alcance 3": r"[Aa]lcance\s*3[^\d]{0,30}([\d.,]+)",
    "MW instalados": r"([\d.,]+)\s*MW",
    "GWh producidos": r"([\d.,]+)\s*GWh",
    "Intensidad emisiones (gCO2/kWh)": r"([\d.,]+)\s*g\s*CO2/kWh",
}

print("Indicadores encontrados en el texto:\n")
for indicador, patron in patrones_esg.items():
    coincidencias = re.findall(patron, texto_completo)
    if coincidencias:
        # Mostramos los primeros 3 valores encontrados
        valores = coincidencias[:3]
        print(f"  📊 {indicador}: {', '.join(valores)}")
    else:
        print(f"  ❌ {indicador}: no encontrado")

### 2.5 Búsqueda contextual: obtener la línea completa

A veces queremos ver no solo el número, sino el contexto. Esto ayuda a interpretar los datos.

In [ ]:
# Buscamos líneas que mencionen conceptos clave de sostenibilidad
conceptos = ["emisiones", "renovable", "CO2", "alcance", "residuo", "biodiversidad"]

lineas = texto_completo.split("\n")

for concepto in conceptos:
    lineas_encontradas = [l.strip() for l in lineas if concepto.lower() in l.lower()]
    print(f"\n🔍 '{concepto}' → {len(lineas_encontradas)} líneas")
    for linea in lineas_encontradas[:2]:  # Solo las 2 primeras
        print(f"   {linea[:120]}")

### 2.6 Comparar indicadores entre informes

La verdadera potencia de la extracción de PDFs aparece cuando comparamos **el mismo indicador entre diferentes informes** (diferentes empresas o diferentes años).

Vamos a crear un extractor genérico que funcione con cualquier informe.

In [ ]:
# Extractor genérico: dado un PDF, busca todos los indicadores ESG

INDICADORES = {
    "emisiones_co2_total": r"emisiones.*?([\d]+[.,]?\d*)\s*(?:millones|Mt|MtCO2)",
    "alcance_1": r"[Aa]lcance\s*1[^\d]{0,40}?([\d]+[.,]?\d*)",
    "alcance_2": r"[Aa]lcance\s*2[^\d]{0,40}?([\d]+[.,]?\d*)",
    "mw_renovable": r"([\d]+[.,]?\d*)\s*MW\s*(?:renovable|verde|limpi|instalad)",
    "produccion_gwh": r"producción.*?([\d]+[.,]?\d*)\s*GWh",
    "intensidad_co2": r"([\d]+[.,]?\d*)\s*g\s*CO2/kWh",
    "empleados": r"(?:plantilla|empleados?|trabajadores?).*?([\d]+[.,]?\d*)",
    "pct_mujeres": r"(?:mujeres|mujer|femenin).*?([\d]+[.,]?\d*)\s*%",
}


# Extraemos texto de un PDF y buscamos indicadores
ruta_pdf = "pdfs_sostenibilidad/iberdrola_gei.pdf"

pdf = pdfplumber.open(ruta_pdf)
texto = "\n".join(p.extract_text() or "" for p in pdf.pages)
pdf.close()

resultados = {}
for nombre_ind, patron in INDICADORES.items():
    match = re.search(patron, texto, re.IGNORECASE)
    if match:
        resultados[nombre_ind] = match.group(1)

print("Indicadores extraídos de Iberdrola GEI 2023:")
for k, v in resultados.items():
    print(f"  {k}: {v}")

In [ ]:
# Si tuviéramos varios informes (ej: Iberdrola, Telefónica, Endesa...)
# haríamos lo mismo para cada uno y crearíamos un DataFrame comparativo:

# Ejemplo de cómo quedaría con datos de varias empresas
datos_comparativa = {
    "empresa": ["Iberdrola", "Telefónica", "Endesa"],
    "emisiones_scope1_tco2": ["extraído PDF", "extraído PDF", "extraído PDF"],
    "emisiones_scope2_tco2": ["extraído PDF", "extraído PDF", "extraído PDF"],
    "pct_energia_renovable": ["extraído PDF", "extraído PDF", "extraído PDF"],
    "mw_renovables": ["extraído PDF", "extraído PDF", "extraído PDF"],
}

print("Estructura del DataFrame comparativo (esquema):")
print(pd.DataFrame(datos_comparativa).to_string(index=False))
print("\n→ Cada celda se rellenaría con los valores reales extraídos de cada PDF")

### 2.7 Extraer tablas de un informe de indicadores

El segundo PDF (indicadores clave) es más compacto y tiene tablas con datos numéricos concretos.

In [ ]:
# Abrimos el PDF de indicadores clave
ruta_indicadores = "pdfs_sostenibilidad/iberdrola_indicadores.pdf"

if os.path.exists(ruta_indicadores):
    pdf2 = pdfplumber.open(ruta_indicadores)
    print(f"Páginas: {len(pdf2.pages)}")
    
    # Extraemos todas las tablas
    tablas_indicadores = []
    for i, pag in enumerate(pdf2.pages):
        for tabla in pag.extract_tables():
            if len(tabla) > 1:  # Al menos cabecera + 1 fila
                df = pd.DataFrame(tabla[1:], columns=tabla[0])
                df["pagina"] = i + 1
                tablas_indicadores.append(df)
                print(f"  Pág {i+1}: tabla de {len(tabla)-1} filas x {len(tabla[0])} cols")
    
    pdf2.close()
    
    # Mostramos la primera tabla extraída
    if tablas_indicadores:
        print("\nPrimera tabla:")
        display(tablas_indicadores[0])
else:
    print(f"Archivo no encontrado: {ruta_indicadores}")
    print("Puede que la descarga haya fallado. Verifica la URL manualmente.")